In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
sns.set_palette('colorblind')
from matplotlib.pyplot import tight_layout
# ##SETTING PARAMS FOR MATPLOTLIB FIGURES
plt.rcParams.update({"figure.figsize": (6, 6),
                 "axes.facecolor": "white",
                 "axes.edgecolor": "black"})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=sns.color_palette('colorblind'))
##set font size
font = {'family': 'sans-serif',
       'weight': 'normal',
       'size': 14}
plt.rc('font', **font)
# ##PANDAS PLOTTING
pd.plotting.register_matplotlib_converters()

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from matplotlib.patches import Patch

# For inline plots in Jupyter
%matplotlib inline
import matplotlib as plt
import glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from tqdm import tqdm
import codecs
import csv
import openpyxl
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Patch

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from statsmodels.discrete.discrete_model import GeneralizedPoisson
import seaborn as sns
from scipy import stats
    

## Step 2 : Import files

In [13]:
data_directory_merged = "/home/paulharford/college/project/project_data/processed/WEATHERED_nas_merged_v2.csv"
full_path_merged = os.path.abspath(data_directory_merged)

In [14]:
df_model = pd.read_csv(full_path_merged)


In [15]:
df_model.isnull().sum()

region                        0
date                          0
fall_category_group           0
callout_count                 0
year                          0
warning_phenomenon            0
warning_severity              0
warning_severity_numeric      0
counties_in_region            0
county_weight                 0
weather_event                 0
population                    0
log_population                0
month                         0
season                        0
is_winter                     0
is_spring                     0
is_summer                     0
is_autumn                     0
callout_count_lag7           42
callout_count_lag30         180
dtype: int64

In [18]:
# Create custom interaction variables for the most significant effects
# 1. Weather-fall category interactions
# 7-day lag (weekly pattern)
df_model['callout_count_lag7'] = df_model.groupby('region')['callout_count'].shift(7)
# 30-day lag (monthly pattern)
df_model['callout_count_lag30'] = df_model.groupby('region')['callout_count'].shift(30)

# Create the design matrices
y, X = patsy.dmatrices(
    "callout_count ~ C(year) + C(month) + C(region) + C(fall_category_group) + "
    "warning_severity:C(fall_category_group) +" 
    "C(region):C(fall_category_group) +"
    "C(year):C(region) +"
    "callout_count_lag30",
    data=df_model,
    return_type='dataframe'
)

# Define the offset - using the indices from y to ensure alignment
offset = df_model.loc[y.index, "log_population"]

### 2. Fit Generalized Poisson Model (For Underdispersion)
print("\n" + "="*60)
print("FITTING GENERALIZED POISSON MODEL (FOR UNDERDISPERSION)")
print("="*60)

# Convert y to 1D array
y_array = np.asarray(y).flatten()

try:
    gp_model = GeneralizedPoisson(endog=y_array, exog=X, offset=offset)
    gp_results = gp_model.fit(maxiter=1000, disp=False)
    
    print("\nGeneralized Poisson Model Summary:")
    print(gp_results.summary())
    gp_fitted = True
except Exception as e:
    print(f"\nGeneralized Poisson model failed: {e}")
    gp_fitted = False


FITTING GENERALIZED POISSON MODEL (FOR UNDERDISPERSION)

Generalized Poisson Model Summary:
                    GeneralizedPoisson Regression Results                     
Dep. Variable:                      y   No. Observations:                63298
Model:             GeneralizedPoisson   Df Residuals:                    63190
Method:                           MLE   Df Model:                          107
Date:                Sun, 06 Jul 2025   Pseudo R-squ.:                  0.1237
Time:                        17:10:55   Log-Likelihood:                -90501.
converged:                       True   LL-Null:                   -1.0328e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------

In [7]:
# Extract all unique regions from your dataset
all_regions = df_final_expanded['region'].unique().tolist()

# Print the regions to verify
print("HSE Regions included in analysis:")
for region in all_regions:
    print(f"- {region}")

HSE Regions included in analysis:
- HSE Dublin and Midlands
- HSE Dublin and North East
- HSE Dublin and South East
- HSE Mid West
- HSE South West
- HSE West and North West
